# Write-up and code for February 1

### To Do
- ~~Work out (in LaTeX) the solution to the Linear Impact model we covered in class~~
- Model a real-world Optimal Trade Order Execution problem as an MDP (with complete order book included in the State)

## Optimal Trade Order Execution
Our task is to sell a large number $N$ of shares, this task must be done within $T$ discrete time steps. Our goal is to maximize Expected Total Utility of Sales Proceeds, where we need to account for both a temporary and a permanent price impact from our market orders. This is a Dynamic Optimization problem that can be modeled as an MDP.
- We have $T$ time steps indexed by $t=1,...,T$
- $P_t$ denotes Bid PRice at start of time step $t$
- $N_t$ denotes number of shares sold in time step $t$
$R_t = N - \sum_{i=1}^{t-1}N_i$ denotes the number of shares remaining to be sold at start of time step $t$. Easy to see that $R_1 = N, N_T = R_T$
-  Price Dynamics are given by $$P_{t+1} = f_t(P_t, N_t, \epsilon_t)$$ where $F_t(\cdot)$ is an arbitrary function representing permanent price impact
- Sales proceeds in time step $t$ is defined as $$N_t\cdot Q_t = N_t \cdot (P_t - g_t(P_t, N-T))$$ where $g_t(\cdot)$ is an arbitrary function representing temporary price impact
- Utility of Sales Proceeds function denoted as $U(\cdot)$

We can formulate this as an MDP:
- The State is a a tuple of $\langle t,P_t, R_t\rangle $, where $1\leq t\leq T$
- Perform Action $N_t$
- Receive Reward $U(N_t\cdot Q_t) = U(N_t\cdot (P_t-g_t(P_t,N_t)))$
- Experience price dynamics $P_{t+1} = f_t(P_t, N_t, \epsilon_t)$

Now we need to make assumptions about the underlying structure of the price dynamics. Consider a simple model with a Linear Price Impact:
- Where $N, N_t, P_t \in \mathbb R$
- Let $P_{t+1} = P_t - \alpha N_t + \epsilon_t$ where $\alpha \in \mathbb R^+$
- The r.v. $\epsilon_t$ is i.i.d. with $\mathbb E[\epsilon_t|N_t,P_t] = 0$
- Temporary price impact is given by $\beta N_t$, so $Q_t = P_t - \beta N_t$ where $\beta \in \mathbb R^+$
- the utility function is the identity function, meaning that we have no. risk-aversion
- MDP Discount factor $\gamma = 1$
The goal is then to find the policy $\pi^*(t,P_t,R_t) = N_t$ that maximizes $$\mathbb E\big[\sum_{t=1}^T N_t\cdot Q_t\big]$$

### Optimal Value Function and Bellman Equation
The value function for policy $\pi$ is defined as $$V^\pi(t,P_t,R_t) = \mathbb E_\pi[\sum_{i=t}^T N_i\cdot(P_i - \beta N_i)~|~(t, P_t,R_t)]$$ 

- The optimal value function is given by $V^*(t,P_t,R_t) = \max_\pi V^\pi(t,P_t,R_t)$
- The optimal value function follows the Bellman Equation

$$V^*(t,P_t,R_t) = \max_{N_t}(N_t(P_t - \beta N_t)+ \mathbb E[V^*(t+1,P_{t+1},R_{t+1})])$$

- At the final time step $T$ we have that $V^*(T,P_T,R_T) = R_T(P_T - \beta R_T)$
- We can use the value function at the last time step, plus the fact that $R_T = R_{T-1} - N_{T-1}$ to find 

$$
\begin{align}
V^*(T-1,P_{T-1},R_{T-1}) & = \max_{N_{T-1}}N_{T-1}(P_{T-1} - \beta N_{T-1}) + \mathbb E[R_T(P_T - \beta R_T)]\\
& = \max_{N_{T-1}}N_{T-1}(P_{T-1} - \beta N_{T-1}) + \mathbb E[(R_{T-1} - N_{T-1})(P_T - \beta (R_{T-1} - N_{T-1}))]\\
& = \max_{N_{T-1}}N_{T-1}(P_{T-1} - \beta N_{T-1}) + (R_{T-1} - N_{T-1})(P_{T-1} - \alpha N_{T-1} - \beta (R_{T-1} - N_{T-1}))\\
\end{align}
$$

- Thus, the value function at $T-1$ is simply a function of $N_{T-1}$, taking the partial derivative w.r.t. $N_{T-1}$ and setting $=0$ gives

$$
(P_{T-1} - 2\beta N^*_{T-1}) - (\alpha  - \beta)R_{T-1} - (P_{T-1} - 2\alpha N^*_{T-1} - \beta (R_{T-1} - 2N^*_{T-1})) = 0\\
\iff\\
- 2\beta N^*_{T-1} - R_{T-1}(\alpha  - \beta) + 2\alpha N^*_{T-1} + \beta(R_{T-1} - 2N^*_{T-1}) = 0\\
\iff\\
2(\alpha - 2\beta)N^*_{T-1} - R_{T-1}(\alpha  - 2\beta) = 0\\
\iff\\
N^*_{T-1} = \frac{R_{T-1}}{2}
$$

- We can then subsitute $N^*_{T-1}$ into the optimal value equation at $T-1$

$$
\begin{align}
V^*(T-1,P_{T-1},R_{T-1}) & = \frac{R_{T-1}}{2}(P_{T-1} - \beta\frac{R_{T-1}}{2}) + (R_{T-1} - \frac{R_{T-1}}{2})(P_{T-1} - \alpha \frac{R_{T-1}}{2} - \beta (R_{T-1} - \frac{R_{T-1}}{2}))\\
& = \frac{R_{T-1}}{2}(P_{T-1} - \beta\frac{R_{T-1}}{2}) + \frac{R_{T-1}}{2}(P_{T-1} - (\alpha + \beta) \frac{R_{T-1}}{2})\\
& = R_{T-1}P_{T-1} - \alpha \frac{R_{T-1}^2}{4} - 2\beta \frac{R_{T-1}^2}{4}\\
& = R_{T-1}P_{T-1} - R_{T-1}^2\frac{\alpha + 2\beta}{4}\\
\end{align}
$$

- If we continue to roll these equations backwards we find that 

$$N^*_{t} = \frac{R_t}{T-t+1}$$

and

$$
V^*(t,P_{t},R_{t}) = R_{t}P_{t} - \frac{R_{t}^2}{2}\cdot\frac{2\beta + (T-t)\alpha}{T-t+1}
$$

- Going back to the initial time step we find that the optimal policy is to sell $\frac{N}{T}$ at every time step, which means that the optimal policy is independent of which state we are in.

### Real World Optimal Execution Problem
We can formulate this as an MDP:
- The State is a a tuple of $\langle t, P_t, R_t, B_t, A_t\rangle $, where $1\leq t\leq T$, $B_t$ is a tuple consisting of a vector $P^{(b)}$ with bid prices, and a vector $N^{(b)}$ with the corresponding order depth. In a similar manner $A_t$ is a tuple consisting of a vector $P^{(a)}$ with ask prices, and a vector $N^{(a)}$ with the corresponding order depth.
- Perform Action $N_t$
- Receive Reward $U(N_t\cdot Q_t) = U(N_t\cdot (P_t-g_t(P_t,N_t)))$
- Experience price dynamics $P_{t+1} = f_t(P_t, N_t, \epsilon_t)$